# Testing of a training pipeline for a single flow condition and a single network architecture

In [ ]:
import sys
import os
from os.path import join
parent_dir = os.path.abspath(join(os.getcwd(), os.pardir))
app_dir = join(parent_dir, "app")
if app_dir not in sys.path:
      sys.path.append(app_dir)

from pathlib import Path
import torch as pt
from utils.TimeSeriesDataset import DataWindowVAE
from utils import Scaler

DATA_PATH = Path(os.path.abspath('')).parent / "data"

#### Load IPSP data

In [ ]:
# load interpolated dataset and pick a flow condition
data = pt.load(join(DATA_PATH, "cp_084_500snaps_interp.pt"))
flow_cond = list(data.keys())[2]
print(flow_cond)
data = data[flow_cond]
data.shape

#### Split and Scale the data

In [ ]:
train_test_split = 0.9
num_timesteps = data.shape[2]
num_train = int(num_timesteps * train_test_split)

# split and scale data
train_data = data[:,:,:num_train]
test_data = data[:, :, num_train:]
scaler = Scaler.MinMaxScaler_1_1().fit(train_data)
train = scaler.scale(train_data)
test = scaler.scale(test_data)
print(train.shape)
print(test.shape)

#### Create dataset for single flow condition

In [ ]:
data_window= DataWindowVAE(train_data, test_data,
                           input_width=10,
                           pred_horizon=5,
                           shift=1,
                           batch_size=32)

train_dataset = data_window.train_dataset
test_dataset = data_window.test_dataset
# print("Timesteps in train dataset:      ", len(train_dataset))
# print("Timesteps in test dataset:       ", len(test_dataset))

print("Shift of the window:             ", data_window.shift)
print("Total window size:               ", data_window.total_window_size)
print("Input width:                     ", data_window.input_width)
print("input indices:                   ", data_window.input_indices)
print("Prediction starts at index:      ", data_window.pred_start)
print("Prediction indices:              ", data_window.pred_indices)
print("Prediction horizon:              ", data_window.pred_horizon)

# data_window.rolling_window()
data_window.rolling_window